In [617]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler
from scipy.stats import skew
from matplotlib.gridspec import GridSpec


In [ ]:
# features = pd.read_csv("/kaggle/input/voice-features/features.csv")
# extra_features = pd.read_csv("/kaggle/input/voice-features/extracted_features_2k_3.csv")
# assert (features.columns == extra_features.columns).all() == True, 'mismatch in features'
# extra_features_v2 = pd.read_csv("/kaggle/input/voice-features-2/extracted_features_3k_v2.csv")
# assert (extra_features_v2.columns == extra_features.columns).all() == True, 'mismatch in features'
features = pd.read_csv("/kaggle/input/voice-features-3/extracted_features_3k_v3.csv")

In [619]:
def get_data_again():
    return pd.concat([features, extra_features])

In [ ]:
#df = get_data_again()
df = features.copy()
df.head()

,duration,wps,f0_mean,f0_std,f0_5_percentile,f0_95_percentile,tempo,formant1,formant2,formant3,...,mfcc_11_skewness,mfcc_12_skewness,cpp_mean,ste_mean,ste_std,ste_max,ste_min,ste_variance,gender,age
0,3.624,2.483444,43.679260,7.371807,34.668420,58.085883,4.966887,684.276613,2144.732066,3551.312656,...,-0.573998,-0.847324,0.078587,0.930276,0.990859,5.593383,0.065408,0.981801,male,twenties
1,8.280,1.690821,53.686158,7.416054,44.160759,67.459444,5.676329,528.719302,1327.511601,2695.891257,...,0.231403,-0.113490,0.080541,1.806639,1.874120,10.059046,0.065434,3.512328,male,twenties
2,3.360,1.190476,41.534685,3.806376,34.448272,47.604511,3.571429,629.261116,1991.730144,3142.621149,...,-0.204671,-0.059365,0.058467,0.223676,0.146010,0.622285,0.044940,0.021319,male,fifties
3,5.472,1.279240,54.116891,12.613461,36.077473,77.781746,3.472222,1730.391551,2606.610018,3787.764806,...,-1.046761,-0.800392,0.046990,1.106869,1.276842,6.991178,0.058588,1.630326,male,fifties
4,5.016,1.993620,63.434174,16.955869,46.759747,89.347764,5.781499,1494.218061,2488.452628,3367.597097,...,-0.219653,-0.237146,0.104711,6.937383,7.923952,62.147619,0.052917,62.789019,male,twenties


In [621]:
df.describe()

,duration,wps,f0_mean,f0_std,f0_5_percentile,f0_95_percentile,tempo,formant1,formant2,formant3,...,mfcc_9_skewness,mfcc_10_skewness,mfcc_11_skewness,mfcc_12_skewness,cpp_mean,ste_mean,ste_std,ste_max,ste_min,ste_variance
count,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,...,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000
mean,4.780975,1.976009,114.212174,98.384737,50.904282,333.737052,5.039624,687.211164,1833.334821,2883.519413,...,-0.441365,-0.267753,-0.347137,-0.254184,0.093265,6.436285,7.667566,39.084639,0.070181,115.753366
std,1.748564,0.530231,148.919355,228.421469,36.991394,661.893918,1.271568,441.433150,545.934084,543.602292,...,0.484152,0.522339,0.460726,0.507711,0.028211,6.755611,7.547738,36.144699,0.025010,302.931014
min,2.016000,0.318066,32.731152,0.000000,32.703196,32.703196,0.345304,53.055715,331.194681,1073.407097,...,-2.721835,-2.717453,-2.370532,-2.583461,-0.022723,0.057446,0.000000,0.057446,0.033304,0.000000
25%,3.456000,1.612103,55.894862,9.790454,36.896821,73.416192,4.201681,395.174270,1467.230770,2536.355263,...,-0.739966,-0.601477,-0.628384,-0.586020,0.074232,1.613715,1.814349,9.625515,0.060510,3.291863
50%,4.584000,1.929012,72.443927,15.971007,45.114880,102.042503,5.050505,534.138272,1796.395020,2838.562609,...,-0.473651,-0.305432,-0.365897,-0.286813,0.089841,5.085478,6.281752,32.436849,0.066101,39.460406
75%,5.904000,2.289377,101.419702,30.814357,58.073297,145.986692,5.853994,818.248114,2160.262578,3201.790421,...,-0.180985,0.051344,-0.084060,0.062437,0.109095,8.821161,10.992443,57.936166,0.073823,120.833811
max,56.184000,7.575758,2720.371358,1867.523201,2400.829293,3898.936015,14.735772,3034.412533,4169.661749,5437.159199,...,2.082980,3.992950,2.612354,2.549911,0.365094,125.336596,86.538039,485.515156,0.755335,7488.832142


In [622]:
def check_nulls(df):
    a = df.isna().sum()
    for index, value in a.items():
        if value > 0:
            return False
    return True

In [623]:
def smart_normalize(df, skew_threshold=0.5):

    numeric_cols = df.select_dtypes(include=['number']).columns
    df_normalized = df.copy()
    transformations = {}
    
    # Step 1: Skewness correction
    for col in numeric_cols:
        col_skew = skew(df[col].dropna())
        if abs(col_skew) > skew_threshold:
            # Apply Yeo-Johnson power transform (handles positive/negative values)
            pt = PowerTransformer(method='yeo-johnson')
            df_normalized[col] = pt.fit_transform(df[[col]]).flatten()
            transformations[col] = {
                'step1': 'power_transform',
                'skewness': col_skew,
                'transformer': pt
            }
        else:
            transformations[col] = {
                'step1': 'none',
                'skewness': col_skew
            }
    
    # Step 2: Standard scaling (applied to all numeric columns)
    scaler = StandardScaler()
    df_normalized[numeric_cols] = scaler.fit_transform(df_normalized[numeric_cols])
    transformations['_standard_scaler'] = scaler
    
    return df_normalized, transformations

In [624]:
def remove_outliers(df, outliers_cols, threshold_percent = 1):
    numeric_cols = df.select_dtypes(include=['number']).columns
    df_clean = df.copy()
    
    for i, (col, states) in enumerate(outliers_cols.items()):
        if states['percent'] < threshold_percent:
            lower_bound = states['lower_bound']
            upper_bound = states['upper_bound']
            
            df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
        
    return df_clean

In [625]:
def find_high_outlier_columns(df):
    outlier_columns = {}
    
    for col in df.select_dtypes(include=np.number).columns:
        if col == 'age' or df[col].nunique() < 10:  
            continue
            
        Q1 = df[col].quantile(0.1)
        Q3 = df[col].quantile(0.9)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
        outlier_percent = (len(outliers) / len(df[col].dropna())) * 100
        
        outlier_columns[col] = {
            'percent': outlier_percent,
            'count': len(outliers),
            'lower_bound': lower_bound,
            'upper_bound': upper_bound
        }
            
    return outlier_columns

In [626]:
def visualize_outliers(df, outlier_columns):
    if not outlier_columns:
        print("No columns with more than 5% outliers found.")
        return
        
    for i, (col, stats) in enumerate(outlier_columns.items()):
        if stats['percent'] > 5:
            plt.figure(figsize=(15, 8))
            gs = GridSpec(2, 2, height_ratios=[3, 1])
            
            # Box plot
            ax1 = plt.subplot(gs[0, 0])
            sns.boxplot(x=df[col], ax=ax1)
            ax1.set_title(f"Box Plot: {col}")
            
            # Histogram with KDE
            ax2 = plt.subplot(gs[0, 1])
            sns.histplot(df[col], kde=True, ax=ax2)
            ax2.axvline(stats['lower_bound'], color='r', linestyle='--', label='Outlier Threshold')
            ax2.axvline(stats['upper_bound'], color='r', linestyle='--')
            ax2.legend()
            ax2.set_title(f"Distribution: {col}")
            
            # Outlier detail table
            ax3 = plt.subplot(gs[1, :])
            ax3.axis('off')
            outlier_text = (
                f"Column: {col}\n"
                f"Outliers: {stats['count']} values ({stats['percent']:.2f}% of non-null data)\n"
                f"Lower bound: {stats['lower_bound']:.2f}\n"
                f"Upper bound: {stats['upper_bound']:.2f}\n"
                f"Min: {df[col].min():.2f}, Max: {df[col].max():.2f}\n"
                f"Mean: {df[col].mean():.2f}, Median: {df[col].median():.2f}"
            )
            ax3.text(0.5, 0.5, outlier_text, ha='center', va='center', fontsize=12)
            
            plt.tight_layout()
            plt.show()
            
            # Print summary
            print(f"\n📊 Outlier Analysis for {col}:")
            print(f"  • {stats['percent']:.2f}% of values are outliers ({stats['count']} out of {len(df[col].dropna())})")
            print(f"  • Outlier thresholds: < {stats['lower_bound']:.2f} or > {stats['upper_bound']:.2f}")
            print("──"*40)

In [655]:
def view_outlier_dist(df, outlier_columns):
    for i, (col, stats) in enumerate(outlier_columns.items()):
        if stats['percent'] > 5:
            df_clean = df.copy()
            df = df_clean[(df_clean[col] >= stats['lower_bound']) & (df_clean[col] <= stats['upper_bound'])]
            
            male_df = df_clean[df_clean['gender'] == 'male']
            female_df = df_clean[df_clean['gender'] == 'female']
                
            male_stats = {'mean': male_df[col].mean(), 'std': male_df[col].std()}
            female_stats = {'mean': female_df[col].mean(), 'std': female_df[col].std()}
            
            male_snr = abs(male_stats['mean']) / male_stats['std'] if male_stats['std'] != 0 else 0
            female_snr = abs(female_stats['mean']) / female_stats['std'] if female_stats['std'] != 0 else 0
            
            pooled_std = np.sqrt((male_stats['std']**2 + female_stats['std']**2)/2)
            cohens_d = abs(male_stats['mean'] - female_stats['mean']) / pooled_std
            print(cohens_d)
            
            plt.figure(figsize=(12, 6))
            
            # Distribution plot
            sns.kdeplot(data=df_clean, x=col, hue='gender', fill=True, alpha=0.3, 
                       common_norm=False, palette={'male':'blue', 'female':'orange'}, hue_order=('male', 'female'))
            
            plt.axvline(male_stats['mean'], color='blue', linestyle='--', 
                        label=f"Male: μ = {male_stats['mean']:.2f}, σ = {male_stats['std']:.2f}")
            plt.axvline(female_stats['mean'], color='orange', linestyle='--',
                        label=f"Female: μ = {female_stats['mean']:.2f}, σ = {female_stats['std']:.2f}")
            
            plt.axvspan(male_stats['mean'] - male_stats['std'], male_stats['mean'] + male_stats['std'], 
                        color='blue', alpha=0.1)
            plt.axvspan(female_stats['mean'] - female_stats['std'], female_stats['mean'] + female_stats['std'],
                       color='orange', alpha=0.1)
            
            plt.title(f"{col}\nCohen's d = {cohens_d:.2f} (Male SNR: {male_snr:.2f}, Female SNR: {female_snr:.2f})")
            plt.legend()
            plt.tight_layout()
            plt.show()
            
            # Print comprehensive comparison
            print(f"\n📊 {col}")
            print(f"   Male: μ/σ = {male_snr:.2f} (μ = {male_stats['mean']:.2f}, σ = {male_stats['std']:.2f})")
            print(f" Female: μ/σ = {female_snr:.2f} (μ = {female_stats['mean']:.2f}, σ = {female_stats['std']:.2f})")
            print(f" Standardized difference (Cohen's d): {cohens_d:.2f}")
            print("──"*30)

In [654]:
def plot_male_female_diff(male_df, female_df):
    for col in male_df.select_dtypes(include=np.number).columns:  # Only numeric columns
        if col == 'gender':
            continue
            
        male_stats = {'mean': male_df[col].mean(), 'std': male_df[col].std()}
        female_stats = {'mean': female_df[col].mean(), 'std': female_df[col].std()}
        
        male_snr = abs(male_stats['mean']) / male_stats['std'] if male_stats['std'] != 0 else 0
        female_snr = abs(female_stats['mean']) / female_stats['std'] if female_stats['std'] != 0 else 0
        
        pooled_std = np.sqrt((male_stats['std']**2 + female_stats['std']**2)/2)
        cohens_d = abs(male_stats['mean'] - female_stats['mean']) / pooled_std
        
        if cohens_d > 0.4:
            plt.figure(figsize=(12, 6))
            
            # Distribution plot
            sns.kdeplot(data=df, x=col, hue='gender', fill=True, alpha=0.3, 
                       common_norm=False, palette={'male':'blue', 'female':'orange'}, hue_order=('male', 'female'))
            
            plt.axvline(male_stats['mean'], color='blue', linestyle='--', 
                        label=f"Male: μ = {male_stats['mean']:.2f}, σ = {male_stats['std']:.2f}")
            plt.axvline(female_stats['mean'], color='orange', linestyle='--',
                        label=f"Female: μ = {female_stats['mean']:.2f}, σ = {female_stats['std']:.2f}")
            
            plt.axvspan(male_stats['mean'] - male_stats['std'], male_stats['mean'] + male_stats['std'], 
                        color='blue', alpha=0.1)
            plt.axvspan(female_stats['mean'] - female_stats['std'], female_stats['mean'] + female_stats['std'],
                       color='orange', alpha=0.1)
            
            plt.title(f"{col}\nCohen's d = {cohens_d:.2f} (Male SNR: {male_snr:.2f}, Female SNR: {female_snr:.2f})")
            plt.legend()
            plt.tight_layout()
            plt.show()
            
            # Print comprehensive comparison
            print(f"\n📊 {col}")
            print(f"   Male: μ/σ = {male_snr:.2f} (μ = {male_stats['mean']:.2f}, σ = {male_stats['std']:.2f})")
            print(f" Female: μ/σ = {female_snr:.2f} (μ = {female_stats['mean']:.2f}, σ = {female_stats['std']:.2f})")
            print(f" Standardized difference (Cohen's d): {cohens_d:.2f}")
            print("──"*30)

In [653]:
def plot_twenties_vs_fifties(twenties_df, fifties_df):
    for col in twenties_df.select_dtypes(include=np.number).columns:  
        if col == 'age':  
            continue
            
        # Calculate statistics
        twenties_stats = {'mean': twenties_df[col].mean(), 'std': twenties_df[col].std()}
        fifties_stats = {'mean': fifties_df[col].mean(), 'std': fifties_df[col].std()}
        
        # Calculate signal-to-noise ratio (mean/std)
        twenties_snr = abs(twenties_stats['mean']) / twenties_stats['std'] if twenties_stats['std'] != 0 else 0
        fifties_snr = abs(fifties_stats['mean']) / fifties_stats['std'] if fifties_stats['std'] != 0 else 0
        
        # Calculate standardized mean difference (Cohen's d)
        pooled_std = np.sqrt((twenties_stats['std']**2 + fifties_stats['std']**2)/2)
        cohens_d = abs(twenties_stats['mean'] - fifties_stats['mean']) / pooled_std
        
        # Only plot if substantial difference exists (Cohen's d > 0.4 effect)
        if cohens_d > 0.2:
            plt.figure(figsize=(12, 6))
            
            # Create temp dataframe with age groups for plotting
            plot_df = pd.concat([
                twenties_df[col].to_frame().assign(age_group='20-29'),
                fifties_df[col].to_frame().assign(age_group='50-59')
            ])
            
            # Distribution plot
            sns.kdeplot(data=plot_df, x=col, hue='age_group', fill=True, alpha=0.3, 
                       common_norm=False, palette={'20-29':'blue', '50-59':'orange'})
            
            # Add statistics annotations
            plt.axvline(twenties_stats['mean'], color='blue', linestyle='--', 
                        label=f"twenties: μ = {twenties_stats['mean']:.2f}, σ = {twenties_stats['std']:.2f}")
            plt.axvline(fifties_stats['mean'], color='orange', linestyle='--',
                        label=f"fifties: μ = {fifties_stats['mean']:.2f}, σ = {fifties_stats['std']:.2f}")
            
            # Add std ranges
            plt.axvspan(twenties_stats['mean'] - twenties_stats['std'], twenties_stats['mean'] + twenties_stats['std'], 
                        color='blue', alpha=0.1)
            plt.axvspan(fifties_stats['mean'] - fifties_stats['std'], fifties_stats['mean'] + fifties_stats['std'],
                       color='orange', alpha=0.1)
            
            plt.title(f"{col}\nCohen's d = {cohens_d:.2f} (twenties SNR: {twenties_snr:.2f}, fifties SNR: {fifties_snr:.2f})")
            plt.legend()
            plt.tight_layout()
            plt.show()
            
            # Print comprehensive comparison
            print(f"\n📊 {col}")
            print(f"   twenties: μ/σ = {twenties_snr:.2f} (μ = {twenties_stats['mean']:.2f}, σ = {twenties_stats['std']:.2f})")
            print(f"   fifties: μ/σ = {fifties_snr:.2f} (μ = {fifties_stats['mean']:.2f}, σ = {fifties_stats['std']:.2f})")
            print(f"   Standardized difference (Cohen's d): {cohens_d:.2f}")
            print("──"*30)

In [652]:
def plot_age_gender_dist(twenties_df, fifties_df):
    for col in twenties_df.select_dtypes(include=np.number).columns:
        if col == 'age':
            continue
            
        # Create subsets for each demographic group
        male_20 = twenties_df[twenties_df['gender'] == 'male'][col].dropna()
        male_50 = fifties_df[fifties_df['gender'] == 'male'][col].dropna()
        female_20 = twenties_df[twenties_df['gender'] == 'female'][col].dropna()
        female_50 = fifties_df[fifties_df['gender'] == 'female'][col].dropna()
        
        # Calculate all Cohen's d comparisons
        def calculate_cohens_d(group1, group2):
            pooled_std = np.sqrt((group1.std()**2 + group2.std()**2)/2)
            return abs(group1.mean() - group2.mean()) / pooled_std
        
        cohens_d = {
            'male_vs_female_20': calculate_cohens_d(male_20, female_20),
            'male_vs_female_50': calculate_cohens_d(male_50, female_50),
            'age_effect_male': calculate_cohens_d(male_20, male_50),
            'age_effect_female': calculate_cohens_d(female_20, female_50)
        }
        
        # Only plot if any comparison shows meaningful effect size
        if any(d > 0.2 for d in cohens_d.values()):
            plt.figure(figsize=(14, 7))
            
            # Create plot dataframe
            plot_df = pd.concat([
                male_20.to_frame().assign(group='Male 20-29'),
                male_50.to_frame().assign(group='Male 50-59'),
                female_20.to_frame().assign(group='Female 20-29'),
                female_50.to_frame().assign(group='Female 50-59')
            ])
            
            # Plot KDE
            palette = {'Male 20-29':'blue', 'Male 50-59':'lightblue',
                     'Female 20-29':'red', 'Female 50-59':'orange'}
            
            for group, color in palette.items():
                group_data = plot_df[plot_df['group'] == group]
                sns.kdeplot(data=group_data, x=col, color=color, 
                            fill=True, alpha=0.2, common_norm=False, 
                            linewidth=2, label=group)
            
            # Add effect size annotations
            text_y = 0.9
            for name, d in cohens_d.items():
                if d > 0.2:
                    comp_name = name.replace('_', ' ').title()
                    plt.text(0.02, text_y, f"{comp_name}: Cohen's d = {d:.2f}",
                            transform=plt.gca().transAxes, fontsize=10,
                            bbox=dict(facecolor='white', alpha=0.7))
                    text_y -= 0.08
            
            plt.title(f"Distribution of {col}\n(Only showing comparisons with Cohen's d > 0.2)")
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.tight_layout()
            plt.show()
            
            # Print detailed statistics
            print(f"\n📊 {col} - Effect Size Analysis:")
            print(f"  Male 20s vs Female 20s: d = {cohens_d['male_vs_female_20']:.2f}")
            print(f"  Male 50s vs Female 50s: d = {cohens_d['male_vs_female_50']:.2f}")
            print(f"  Age Effect (Male 20s vs 50s): d = {cohens_d['age_effect_male']:.2f}")
            print(f"  Age Effect (Female 20s vs 50s): d = {cohens_d['age_effect_female']:.2f}")
            print("──"*40)

In [657]:
def duration_plot(male_df, female_df):
    # Calculate statistics
    twenties_stats = {'mean': male_df['duration'].mean(), 'std': male_df['duration'].std()}
    fifties_stats = {'mean': female_df['duration'].mean(), 'std': female_df['duration'].std()}
    
    # Calculate signal-to-noise ratio (mean/std)
    twenties_snr = abs(twenties_stats['mean']) / twenties_stats['std'] if twenties_stats['std'] != 0 else 0
    fifties_snr = abs(fifties_stats['mean']) / fifties_stats['std'] if fifties_stats['std'] != 0 else 0
    
    # Calculate standardized mean difference (Cohen's d)
    pooled_std = np.sqrt((twenties_stats['std']**2 + fifties_stats['std']**2)/2)
    cohens_d = abs(twenties_stats['mean'] - fifties_stats['mean']) / pooled_std
    
    plt.figure(figsize=(12, 6))
    
    # Create temp dataframe with age groups for plotting
    plot_df = pd.concat([
        twenties_df['duration'].to_frame().assign(age_group='20-29'),
        fifties_df['duration'].to_frame().assign(age_group='50-59')
    ])
    
    # Distribution plot
    sns.kdeplot(data=plot_df, x='duration', hue='age_group', fill=True, alpha=0.3, 
               common_norm=False, palette={'20-29':'blue', '50-59':'orange'})
    
    # Add statistics annotations
    plt.axvline(twenties_stats['mean'], color='blue', linestyle='--', 
                label=f"twenties: μ = {twenties_stats['mean']:.2f}, σ = {twenties_stats['std']:.2f}")
    plt.axvline(fifties_stats['mean'], color='orange', linestyle='--',
                label=f"fifties: μ = {fifties_stats['mean']:.2f}, σ = {fifties_stats['std']:.2f}")
    
    # Add std ranges
    plt.axvspan(twenties_stats['mean'] - twenties_stats['std'], twenties_stats['mean'] + twenties_stats['std'], 
                color='blue', alpha=0.1)
    plt.axvspan(fifties_stats['mean'] - fifties_stats['std'], fifties_stats['mean'] + fifties_stats['std'],
               color='orange', alpha=0.1)
    
    plt.title(f"{col}\nCohen's d = {cohens_d:.2f} (twenties SNR: {twenties_snr:.2f}, fifties SNR: {fifties_snr:.2f})")
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Print comprehensive comparison
    print(f"\n📊 duration")
    print(f"   twenties: μ/σ = {twenties_snr:.2f} (μ = {twenties_stats['mean']:.2f}, σ = {twenties_stats['std']:.2f})")
    print(f"   fifties: μ/σ = {fifties_snr:.2f} (μ = {fifties_stats['mean']:.2f}, σ = {fifties_stats['std']:.2f})")
    print(f"   Standardized difference (Cohen's d): {cohens_d:.2f}")
    print("──"*30)

In [658]:
def analyze_feature_correlations(df, threshold=0.5):
    
    numeric_df = df.select_dtypes(include=['number'])
    
    corr_matrix = numeric_df.corr()
    
    # Find high correlations
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr.append((corr_matrix.columns[i], corr_matrix.columns[j], 
                                 corr_matrix.iloc[i, j]))
    return high_corr

In [643]:
def features_to_keep(all_features, corr_features, to_be_removed):
    features_set = set()
    to_keep = []
    for removed in to_be_removed:
        features_set.add(removed)
    
    for x, y, z in corr_features:
        if x not in features_set and y not in features_set:
            to_keep.append(x)
            features_set.add(x)
            features_set.add(y)

    for feature in all_features:
        if feature not in features_set:
            to_keep.append(feature)

    return to_keep

## Calls

In [ ]:
outliers_cols = find_high_outlier_columns(df)
visualize_outliers(df, outliers_cols)

In [ ]:
view_outlier_dist(df, outliers_cols)

In [ ]:
assert check_nulls(df) == True, 'there are nulls'

In [ ]:
df_normalized, transformations = smart_normalize(df)
outliers_cols = find_high_outlier_columns(df_normalized)
df_clean = remove_outliers(df_normalized, outliers_cols)
df = df_clean.copy()

In [ ]:
male_df = df[df['gender'] == 'male']
female_df = df[df['gender'] == 'female']
twenties_df = df[df['age'] == 'twenties']
fifties_df = df[df['age'] == 'fifties']

In [ ]:
correlated_features = analyze_feature_correlations(df, 0.8)

In [ ]:
remaining_features = features_to_keep(df.select_dtypes(include=['number']).columns, correlated_features, to_be_removed=['duration'])
remaining_features.append('age')
remaining_features.append('gender')

In [ ]:
assert len(remaining_features) == len(set(remaining_features)), 'problem in feature selection'
df = df[remaining_features]

## Baseline Model

In [647]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA

In [659]:
def baseline(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    return clf.feature_importances_

In [648]:
X = df.drop(['gender', 'age'], axis=1)
y = df['gender'].map({'male' : 1, 'female' : 0})

In [ ]:
baseline(X, y)

In [347]:
pca = PCA(n_components=90)  
X_pca = pca.fit_transform(X)  

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(range(pca.n_components_), pca.explained_variance_ratio_)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Explained Variance')
plt.show()

cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.plot(cumulative_variance, marker='o')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.legend()
plt.show()

In [ ]:
baseline(X, y)

In [660]:
X = df.drop(['gender', 'age'], axis=1)
y = df['age'].map({'twenties' : 1, 'fifties' : 0})
baseline(X, y)

Accuracy: 0.809376927822332
              precision    recall  f1-score   support

           0       0.89      0.31      0.46       424
           1       0.80      0.99      0.88      1197

    accuracy                           0.81      1621
   macro avg       0.84      0.65      0.67      1621
weighted avg       0.82      0.81      0.77      1621



array([0.01664363, 0.00954939, 0.00891846, 0.00904001, 0.00835281,
       0.01066492, 0.01475244, 0.01434401, 0.00871514, 0.0080285 ,
       0.00649238, 0.00795689, 0.00748534, 0.00689819, 0.00742955,
       0.01130259, 0.0104755 , 0.00953572, 0.01706601, 0.02395911,
       0.00893129, 0.00718669, 0.00790893, 0.00836331, 0.01269263,
       0.0228777 , 0.01369332, 0.01146301, 0.01642351, 0.01113599,
       0.01070783, 0.02036954, 0.00988634, 0.01197812, 0.01248607,
       0.00958982, 0.01145441, 0.01150285, 0.01487449, 0.00891717,
       0.00932225, 0.00874749, 0.01054487, 0.0106987 , 0.00835294,
       0.01008452, 0.00940933, 0.00698448, 0.0075156 , 0.00797834,
       0.00719481, 0.00735475, 0.00655877, 0.00622104, 0.00724811,
       0.00704689, 0.00782037, 0.00747624, 0.01069299, 0.01072026,
       0.00709258, 0.00862228, 0.00711858, 0.00765698, 0.00864683,
       0.00600511, 0.00694956, 0.00827101, 0.00698905, 0.00654758,
       0.00768828, 0.00831488, 0.0086422 , 0.00863568, 0.00856

In [ ]:
df.to_csv('/kaggle/working/cleaned.csv')